### O METAR, sigla de Meteorological Aerodrome Report, é o informe meteorológico regular de aeródromo e contém as seguintes informações na sequência:

- Grupos de identificação;  
- Vento à superfície;  
- Visibilidade horizontal; 
- Alcance visual na pista (quando houver); 
- Tempo presente; 
- Nuvens (ou visibilidade vertical, se for o caso); 
- Temperaturas do ar e do ponto de orvalho; 
- Pressão atmosférica (QNH); e  
- Informações suplementares de inclusão condicional sobre tempo recente, cortante do vento, temperatura da superfície do mar, estado do mar e, por Acordo Regional de Navegação Aérea, o estado da pista.

https://ajuda.decea.mil.br/base-de-conhecimento/como-decodificar-o-metar-e-o-speci/


pip install python-metar


In [107]:
from metar import Metar
from datetime import datetime, timedelta
import pandas as pd

df_metar = pd.read_csv("./processed_data/metar_silver.csv")



In [118]:


def interpretaMetaR(metarBruto):

    metarBruto = metarBruto.replace('METAF', 'METAR')
    metarBruto = metarBruto.replace('\n', '')


    station = ''
    type = ''
    hora_formated = ''
    station = ''
    dew_point = ''
    windDirection = ''
    windVelocity = ''
    visibility = ''
    pressure = ''
    weather = ''
    sky = ''
    skyFeet= ''

    try: 

        obs = Metar.Metar(metarBruto[:-1])
        print(obs)

        for item in obs.string().split('\n'):
            if("station" in item):
                station = item.split(" ")[1]
            if("type" in item):
                type = " ".join(item.split(" ")[1:])
            if("time" in item):
                timeZ = " ".join(item.split(" ")[1:])
                data_obj = datetime.strptime(timeZ, "%a %b %d %H:%M:%S %Y")
                nova_data_obj = data_obj - timedelta(hours=3)
                hora_formated = nova_data_obj.strftime("%a %b %d %H:%M:%S %Y")
            if("temperature" in item):
                station = item.split(" ")[1]
            if("dew point" in item):
                dew_point = item.split(" ")[2]
            if("wind" in item):
                wind = (item.split(" ")[1:])
                windDirection = wind[0]
                windVelocity = wind[2]
            if("visibility" in item):
                visibility = item.split(" ")[1]
            if("pressure" in item):
                pressure = item.split(" ")[1]
            if("weather" in item):
                weather = " ".join(item.split(" ")[1:])
            if("sky" in item):
                
                sky = item.split(" ")[1]
                skyFeet = item.split(" ")[3]
    except:
        pass

    return station, type, hora_formated, station,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet
        



In [109]:
resultado_df =  df_metar['metaf'].apply(interpretaMetaR)
resultado_df =  pd.DataFrame(resultado_df.tolist(),columns=['station', 'type', 'hora_formated', 'station','dew_point','windDirection','windVelocity','visibility','pressure','weather','sky','skyFeet'] )


metaR_silver = pd.concat([df_metar, resultado_df], axis=1)

In [112]:
metaR_silver.metaf.iloc[30710]

'METAF SBSP 012000Z  13007KT 9999  -TSRA  SCT025 FEW030CB 26/19 Q1014=\n'

In [119]:
interpretaMetaR("METAF SBSP 012000Z  13007KT 9999  -TSRA  SCT025 FEW030CB 26/19 Q1014=\n")

station: SBSP
type: routine report, cycle 20 (automatic report)
time: Fri Sep  1 20:00:00 2023
temperature: 26.0 C
dew point: 19.0 C
wind: SE at 7 knots
visibility: greater than 10000 meters
pressure: 1014.0 mb
weather: light thunderstorm with rain
sky: scattered clouds at 2500 feet
     a few cumulonimbus at 3000 feet
METAR: METAR SBSP 012000Z  13007KT 9999  -TSRA  SCT025 FEW030CB 26/19 Q1014


('26.0',
 'routine report, cycle 20 (automatic report)',
 'Fri Sep 01 17:00:00 2023',
 '26.0',
 '19.0',
 'SE',
 '7',
 'greater',
 '1014.0',
 'light thunderstorm with rain',
 'scattered',
 'at')